In [1]:
import base64
import json
import pandas as pd
import requests
import tensorflow as tf

In [8]:
import pandas as pd
df = pd.read_csv(r'D:\DOWNLOAD\a443-MLOps-ML-Pipeline\a443-MLOps-ML-Pipeline\data\Student.csv')
df.head()

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0


In [12]:
gpa_value = df["GPA"][0]

gpa_value

2.929195591667681

In [13]:
import json
import requests
import tensorflow as tf
import base64

def prepare_json(gpa):
    feature_spec = {
        "GPA": tf.train.Feature(
            float_list=tf.train.FloatList(value=[gpa])
        )
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps({"signature_name": "serving_default", "instances": result})


def make_predictions(gpa):
    json_data = prepare_json(gpa)

    endpoint = "http://localhost:8080/v1/models/tutor-detection-model:predict"
    response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})

    if response.status_code == 200:
        predictions = response.json()["predictions"][0][0]
        if predictions >= 0.5:
            prediction_label = "Tutor"
        else:
            prediction_label = "Not Tutor"
        return {
            "prediction": predictions,
            "label": prediction_label
        }
    else:
        return {"error": "Failed to get predictions from the model."}



In [ ]:
# Example usage
result = make_predictions(gpa_value)
print(result)

In [17]:
# Actual label
actual_label = df["Tutoring"][0]
print("Actual Label:", actual_label)

Actual Label: 1
